## Lazy-PRM-Varianten zielen auf Single-Query-Anfragen ab. Welches andere Verfahren zielt auf Multi-Query-Anfragen ab? Was sind Vor- und Nachteile dieses Verfahrens und warum kann es insbesondere für Multi-Query-Aufgabenstellungen gut sein?


## Unterschied Single-Query vs. Multi-Query
Der Unterschied liegt darin, wie oft und unter welchen Bedingungen wir einen Pfad in einer unveränderten Umgebung suchen.
1. Single-Query (Einmalige Anfrage)
Hier geht es darum, einen einzigen Pfad von einem Start $A$ zu einem Ziel $B$ so schnell wie möglich zu finden. Wir gehen davon aus, dass wir die berechneten Daten danach wahrscheinlich verwerfen müssen, weil sich die Hindernisse bewegen oder sich die Aufgabe komplett ändert.
Genau das mach der Lazy-PRM. Er pürft nur auf Kollisionen von Knoten, die Bestandteil des Lösungspfades sein könnten und "ignoriert" den restlichen Arbeitsraum, der für die Aufgabe nicht relevant ist.

    Ein Anwendungs-Beispiel wäre ein Roboter, der Teile aus einer Kiste greift ("Bin Picking"). Die Teile liegen jedes Mal anders und die "Karte" des Arbeitsraums ändert sich mit jedem Griff. Eine feste Roadmap wäre hier sinnlos, da sie sofort veraltet wäre.

2. Multi-Query (Mehrfache Anfrage)
Hierbei bleibt die Umgebung (Hindernisse) statisch, die "Karte" des Arbeitsraumes ist immer die gleiche. Über einen langen Zeitraum sollen viele verschiedene Pfade zwischen unterschiedlichen Start- und Zielpunkten in diesem Raum gefunden werden.
Um diese Anforderung zu implementieren wird zu Beginn viel Rechenzeit investiert, um eine "perfekte" Karte (Roadmap) des gesamten Freiraums zu erstellen. Zukünftige Pfadsuchen sind dann extrem schnell, weil nur noch im Graphen gesucht werden muss und keine zusätzlichen Kollisionschecks mehr ausgeführt werden müssen.

    Ein Anwendungs-Beispiel wäre hier ein Schweißroboter in der Automobilindustrie. Die Karosserie ist fest eingespannt (Hindernis bewegt sich nicht). Der Roboter muss aber nacheinander 50 verschiedene Schweißpunkte an der Karosserie anfahren. Man berechnet einmal die Roadmap um die Karosserie herum und kann dann blitzschnell von Punkt 1 zu Punkt 2 etc. planen.

## Verfahren für Multi-Query Anfragen, Vor- und Nachteile
Das klassische Gegenstück ist das Standard-PRM (Probabilistic Roadmap).
Hierbei wird strikt zwischen einer Construction Phase (Aufbau der Roadmap) und einer Query Phase (Abfrage) getrennt. In der Construction Phase wird die komplette Roadmap erstellt und alle Knoten sowie Kanten werden initial auf Kollisionsfreiheit geprüft, bevor überhaupt eine Start-Ziel-Anfrage kommt.

1. Vorteile:
    - Extrem schnelle Abfragen: Da die Roadmap bereits validiert ist, muss bei einer Anfrage (Query) nur noch ein Graph-Suchalgorithmus (wie A* oder Dijkstra) ausgeführt werden. Das dauert meist nur Millisekunden.
    - Wiederverwendbarkeit: Einmal berechnet, kann die Roadmap für hunderte verschiedene Start-Ziel-Konfigurationen im selben Arbeitsraum genutzt werden.
2. Nachteile:
    - Lange Initialdauer: Der Aufbau dauert lange, da viele Kollisionschecks durchgeführt werden, die für eine spezifische Pfadanfrage vielleicht gar nicht nötig wären (viele Bereiche des Arbeitsraums werden geprüft, die man nie durchquert).
    - Speicherbedarf: Bei komplexen Umgebungen muss die Roadmap sehr dicht sein, was viel Speicher kostet.
    - Sobald ein Hindernis verschoben wird muss die gesamte Roadmap neu berechnet werden

Man nutzt dieses Verfahren für Multi-Query Aufgaben, da sich die hohen initialen Berechnungskosten über die Anzahl der Anfragen ausgleichen. Wenn ein Roboter in einer festen Zelle immer wieder neue Teile greifen muss, rechnet man einmalig (z.B. über Nacht) die Roadmap aus und kann im laufenden Betrieb verzögerungsfrei planen.

## Mit welchen Bewegungsbefehlen würden Sie den Ergebnispfad des Lazy-PRM in ein Roboterprogramm umwandeln? Erläutern Sie, warum das zwar möglich ist, aber es nicht unbedingt sinnvoll ist.

Man würde PTP-Befehle (Point-to-Point) verwenden (in vielen Robotersprachen z.B. MOVJ, PTP oder MoveJ).
Der Lazy-PRM liefert eine Liste von validen Gelenkwinkel-Konfigurationen (Knoten) im C-Space (Konfigurationsraum). Eine Kante im PRM-Graphen entspricht der direkten, linearen Interpolation zwischen zwei Winkelstellungen. Genau das führt ein PTP-Befehl physikalisch aus: Alle Achsen bewegen sich synchron von Konfiguration A nach Konfiguration B. Da der Planer sichergestellt hat, dass diese Linie im C-Space kollisionsfrei ("C-free") ist, kann der Roboter die Bewegung sicher ausführen.
    
Möglich ist es da der Pfad ist geometrisch validiert und kollisionsfrei ist. Der Roboter kann ihn einfach abfahren.
Allerdings ist es kein idealer Pfad, da die Punkte zufällig gesampelt wurden. Der Roboter müsste an jedem Knoten abbremsen und neu beschleunigen (Stop-and-Go), was extrem langsam ist und die Mechanik (Getriebe) stark belastet.
Außerdem wird der Roboter oft Umwege machen, statt "direkt" zu fahren, da er an die zufällige Verteilung der Knoten gebunden ist.

## Was müssen Sie machen, damit er sinnvoll (schnell) abgefahren werden kann? Erläutern Sie die Vorgehensweise (nicht programmieren).

Damit der vom Lazy-PRM berechnete Roh-Pfad sinnvoll und effizient vom Roboter abgefahren werden kann, ist ein anschließendes Post-Processing – auch Bahnglättung oder Path Smoothing genannt – zwingend erforderlich. Der ursprüngliche Pfad enthält durch das zufällige Sampling oft unnötige Zick-Zack-Bewegungen und Artefakte, die bereinigt werden müssen.Hierfür kommt primär die sogenannte Shortcut-Heuristik (Pruning) zum Einsatz. Das Verfahren basiert auf der Idee, den Pfad iterativ zu straffen, ähnlich einem Gummiband, das sich zusammenzieht. Dabei werden wiederholt zwei nicht direkt benachbarte Knoten des Pfades ausgewählt (beispielsweise Knoten $A$ und $E$) und mittels eines "Line-of-Sight Checks" überprüft, ob ihre direkte Verbindungslinie kollisionsfrei ist. Ist dies der Fall, werden alle dazwischenliegenden Knoten ($B, C, D$) entfernt und durch die direkte Abkürzung ersetzt. Dieser Vorgang wird so lange wiederholt, bis keine wesentlichen Verkürzungen mehr möglich sind.Um im Anschluss das mechanisch belastende und zeitraubende "Stop-and-Go" an den verbleibenden scharfen Ecken zu vermeiden, werden diese geglättet. Durch das sogenannte Überschleifen (Blending) oder den Einsatz von Splines fährt der Roboter die Eckpunkte nicht exakt an, sondern beschreibt eine Kurve um sie herum. Dies ermöglicht eine flüssige Bewegung mit konstanter, hoher Geschwindigkeit ohne unnötige Stopps.